In [ ]:
from pyspark.sql import SparkSession

# Initialize SparkSession
spark = SparkSession \
    .builder \
    .appName("DatawarehouseSpark Application") \
    .getOrCreate()
 
inputFileHire ='/home/mose/Downloads/CSVS/client_hiring_dt.csv' #"/user/user17/mosedata_proj/input/client_hiring_dt.csv"
#inputFileBio ='/home/mose/Downloads/CSVS/client_bio_dt.csv'  #"/user/user17/mosedata_proj/input/client_bio_dt.csv"
#inputFileCom = '/home/mose/Downloads/CSVS/client_communication_dt.csv' #"/user/user17/mosedata_proj/input/client_communication_dt.csv"
#inputFileAct = '/home/mose/Downloads/CSVS/client_activities_dt.csv' #"/user/user17/mosedata_proj/input/client_activities_dt.csv"
#inputFileFact = '/home/mose/Downloads/CSVS/client_fact_ft.csv' #"/user/user17/mosedata_proj/input/client_fact_ft.csv"

 
# Create DataFrame from CSV file
dfM =  spark.read.format("csv").option("sep", ",").option("inferSchema", "true").option("header", "true").load(inputFileHire)
#dfR =  spark.read.format("csv").option("sep", ",").option("inferSchema", "true").option("header", "true").load(inputFileBio)
#dfCom =  spark.read.format("csv").option("sep", ",").option("inferSchema", "true").option("header", "true").load(inputFileCom)
#dfAct =  spark.read.format("csv").option("sep", ",").option("inferSchema", "true").option("header", "true").load(inputFileAct)
#dfFact =  spark.read.format("csv").option("sep", ",").option("inferSchema", "true").option("header", "true").load(inputFileFact)


# Print the schema of the DataFrames
dfM.printSchema()
dfR.printSchema()
dfCom.printSchema()
dfAct.printSchema()
dfFact.printSchema()


#dfR.groupBy('movieId').count().show(50)
#dfM.where("director='Martin Brest'").join(dfR,"movieId").select("movieId","director","user_name","rating").show(50)


# Stop the Spark Session
spark.stop()


In [13]:
%%writefile dataframes.py
from pyspark.sql import SparkSession

# Initialize SparkSession
spark = SparkSession \
    .builder \
    .appName("DatawarehouseSpark Application") \
    .getOrCreate()

#Input the data
inputFileHire ='/user/user17/mosedata_proj/input/client_hiring_dt.csv'
inputFileBio ='/user/user17/mosedata_proj/input/client_bio_dt.csv'
inputFileCom = '/user/user17/mosedata_proj/input/client_communication_dt.csv'
inputFileAct = '/user/user17/mosedata_proj/input/client_activities_dt.csv'
inputFileFact = '/user/user17/mosedata_proj/input/client_fact_ft.csv'

#Create data frames from sources. Tables were converted to csvs now being converted to dataframes
dfHire =  spark.read.format("csv").option("sep", ",").option("inferSchema", "true").option("header", "true").load(inputFileHire)
dfBio =  spark.read.format("csv").option("sep", ",").option("inferSchema", "true").option("header", "true").load(inputFileBio)
dfCom =  spark.read.format("csv").option("sep", ",").option("inferSchema", "true").option("header", "true").load(inputFileCom)
dfAct =  spark.read.format("csv").option("sep", ",").option("inferSchema", "true").option("header", "true").load(inputFileAct)
dfFact =  spark.read.format("csv").option("sep", ",").option("inferSchema", "true").option("header", "true").load(inputFileFact)

# Print the schema of the DataFrames
dfHire.printSchema() 
dfBio.printSchema()
dfCom.printSchema()
dfAct.printSchema()
dfFact.printSchema()

#Clients from different service branches
dfBio.groupBy('service_branch__c').count().show()

#Types of jobs the guys got hired in
dfHire.groupBy('job_function_hired_in__c').count().show()

#Temporary objects from which sql statements are run for each dataframe
dfHire.createOrReplaceTempView("dfHire_sql")
dfBio.createOrReplaceTempView("dfBio_sql")
dfCom.createOrReplaceTempView("dfCom_sql")
dfAct.createOrReplaceTempView("dfAct_sql")
dfFact.createOrReplaceTempView("dfFAct_sql")

#Mixed information from all 5 tables of the database
spark.sql(''' Select hires.hired, bio.service_rank__c , 
              bio.service_branch__c , fact.yearsinservice, fact.reg_afterservice_years,
              com.responsive__c, act.finalized_hhusa_revised_resume_on_file__c as resume_done
              from dfHire_sql hires
              inner join dfBio_sql bio on bio.id = hires.id 
              inner join dfFAct_sql fact on fact.id = hires.id
              inner join dfCom_sql com on com.id = hires.id
              inner join dfAct_sql act on act.id = hires.id
              where hires.hired = "1"
              
              ''').show()

#Account create date by month of the year
spark.sql( """SELECT count(id) , 
              Extract(Month From create_ddate) as month 
              From  dfAct_sql  
              group by month 
              ORDER BY month ASC """).show()


#Registration for services by year
spark.sql( """SELECT count(id) , 
Extract(Year From create_ddate) as year_registered 
              From  dfAct_sql  
              group by year_registered
              ORDER BY year_registered ASC """).show()

#Hired by Service branch
spark.sql(""" Select count(hires.id) as hired,bio.service_branch__c 
              From dfHire_sql hires 
              inner join dfBio_sql bio on bio.id = hires.id
              Where hires.hired = 1
              group by bio.service_branch__c
              order by hired asc
              """).show()

#Not hired by Service Branch
spark.sql(""" Select count(hires.id) as Not_hired,bio.service_branch__c 
              From dfHire_sql hires 
              inner join dfBio_sql bio on bio.id = hires.id
              Where hires.hired = 0
              group by bio.service_branch__c
              order by Not_hired asc
              """).show()

#Hired by rank
spark.sql(""" Select count(hires.id) as hired,bio.service_rank__c
              From dfHire_sql hires 
              inner join dfBio_sql bio on bio.id = hires.id
              Where hires.hired = 0
              group by bio.service_rank__c
              order by hired desc
              """).show()

#Prefered method of contact for the hired
spark.sql(""" select count(com.id) as count, com.preferred_method_of_contact__c 
              from dfCom_sql com 
              inner join dfHire_sql hires on com.id = hires.id
              where hires.hired = 1
              group by com.preferred_method_of_contact__c 
              order by count
          """).show()

#Prefered method of contact for the Unhired
spark.sql(""" select count(com.id) as count, com.preferred_method_of_contact__c 
              from dfCom_sql com 
              inner join dfHire_sql hires on com.id = hires.id
              where hires.hired = 0
              group by com.preferred_method_of_contact__c 
              order by count
          """).show()

Overwriting dataframes.py


In [1]:
from pyspark.sql import SparkSession

# Initialize SparkSession
spark = SparkSession \
    .builder \
    .appName("DatawarehouseSpark Application") \
    .getOrCreate()

In [2]:
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.sql.types import StringType
from pyspark.sql.functions import col, regexp_replace, split
from pyspark.ml.feature import Word2Vec
from pyspark.ml.feature import FeatureHasher
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import RandomForestClassifier
from datetime import datetime
from pyspark.ml.clustering import KMeans
from pyspark.sql.types import *
from pyspark.ml import Pipeline
import copy

In [3]:
inputFileHire ='/home/mose/Downloads/CSVS/client_hiring_dt.csv' #"/user/user17/mosedata_proj/input/client_hiring_dt.csv"
inputFileBio ='/home/mose/Downloads/CSVS/client_bio_dt.csv'  #"/user/user17/mosedata_proj/input/client_bio_dt.csv"
inputFileCom = '/home/mose/Downloads/CSVS/client_communication_dt.csv' #"/user/user17/mosedata_proj/input/client_communication_dt.csv"
inputFileAct = '/home/mose/Downloads/CSVS/client_activities_dt.csv' #"/user/user17/mosedata_proj/input/client_activities_dt.csv"
inputFileFact = '/home/mose/Downloads/CSVS/client_fact_ft.csv' #"/user/user17/mosedata_proj/input/client_fact_ft.csv"

In [4]:
# Create DataFrame from CSV file
dfHire =  spark.read.format("csv").option("sep", ",").option("inferSchema", "true").option("header", "true").load(inputFileHire)
dfBio =  spark.read.format("csv").option("sep", ",").option("inferSchema", "true").option("header", "true").load(inputFileBio)
dfCom =  spark.read.format("csv").option("sep", ",").option("inferSchema", "true").option("header", "true").load(inputFileCom)
dfAct =  spark.read.format("csv").option("sep", ",").option("inferSchema", "true").option("header", "true").load(inputFileAct)
dfFact =  spark.read.format("csv").option("sep", ",").option("inferSchema", "true").option("header", "true").load(inputFileFact)

In [5]:
# Print the schema of the DataFrames
dfHire.printSchema() 
dfBio.printSchema()
dfCom.printSchema()
dfAct.printSchema()
dfFact.printSchema()

root
 |-- id: string (nullable = true)
 |-- dsubimittedforhire: string (nullable = true)
 |-- dconfirmedforhire: string (nullable = true)
 |-- difsubmitconfirmed: integer (nullable = true)
 |-- startdate: string (nullable = true)
 |-- datdifconfirstart: integer (nullable = true)
 |-- job_type__c: string (nullable = true)
 |-- job_function_hired_in__c: string (nullable = true)
 |-- hiring_account__c: string (nullable = true)
 |-- hire_heroes_usa_confirmed_hire__c: integer (nullable = true)
 |-- hired_location__c: string (nullable = true)
 |-- hired_zip_code__c: string (nullable = true)
 |-- industry_hired_in__c: string (nullable = true)
 |-- hired_but_still_active_and_looking__c: string (nullable = true)
 |-- hired: string (nullable = true)
 |-- client_hiring_dt_id: string (nullable = true)

root
 |-- id: string (nullable = true)
 |-- client__c: integer (nullable = true)
 |-- client_type__c: string (nullable = true)
 |-- service_rank__c: string (nullable = true)
 |-- service_branch__c: 

In [6]:
#Clients from different service branches
dfBio.groupBy('service_branch__c').count().show()

+-----------------+-----+
|service_branch__c|count|
+-----------------+-----+
|          Marines|13691|
|           Spouse|    2|
|  Merchant Marine|    1|
|             null|37153|
|        Air Force|13416|
|             Army|51770|
|      Coast Guard|  837|
|   Not Applicable|    4|
|             Navy|15567|
+-----------------+-----+



In [7]:
#Types of jobs the guys got hired in
dfHire.groupBy('job_function_hired_in__c').count().show()

+------------------------+------+
|job_function_hired_in__c| count|
+------------------------+------+
|    IT - Help Desk/Su...|   287|
|    Skilled Labor/Trades|   484|
|    Purchasing/Procur...|    73|
|                   Sales|   697|
|    Firefighter/EMT/E...|    71|
|                 Science|    28|
|    Training/Instruct...|  1113|
|    Supply Chain/Logi...|   892|
|             Engineering|   465|
|    Media/Journalism/...|    54|
|         Banking/Finance|   522|
|              Healthcare|   900|
|    Quality Assurance...|   238|
|              Accounting|   171|
|    IT - Computer Sci...|    41|
|                    null|109255|
|    Business Development|   196|
|    Facilities Manage...|   165|
|              Consultant|   456|
|    Installation/Main...|  1223|
+------------------------+------+
only showing top 20 rows



In [8]:
#Temporary objects from which sql statements are run for each dataframe
dfHire.createOrReplaceTempView("dfHire_sql")
dfBio.createOrReplaceTempView("dfBio_sql")
dfCom.createOrReplaceTempView("dfCom_sql")
dfAct.createOrReplaceTempView("dfAct_sql")
dfFact.createOrReplaceTempView("dfFAct_sql")

In [9]:
#Mixed information from all 5 tables of the database
spark.sql(''' Select hires.hired, bio.service_rank__c , 
              bio.service_branch__c , fact.yearsinservice, fact.reg_afterservice_years,
              com.responsive__c, act.finalized_hhusa_revised_resume_on_file__c as resume_done
              from dfHire_sql hires
              inner join dfBio_sql bio on bio.id = hires.id 
              inner join dfFAct_sql fact on fact.id = hires.id
              inner join dfCom_sql com on com.id = hires.id
              inner join dfAct_sql act on act.id = hires.id
              where hires.hired = "1"
              
              ''').show()

+-----+---------------+-----------------+--------------+----------------------+-------------+-----------+
|hired|service_rank__c|service_branch__c|yearsinservice|reg_afterservice_years|responsive__c|resume_done|
+-----+---------------+-----------------+--------------+----------------------+-------------+-----------+
|    1|            O-3|             Army|         11.04|                 -0.33|            1|          1|
|    1|            E-9|             Army|         30.18|                 -0.15|            1|          1|
|    1|            E-4|             Army|           9.8|                 -0.73|            1|          1|
|    1|            E-4|             Army|          2.81|                  0.43|            1|          1|
|    1|            E-4|          Marines|          4.47|                  4.45|            1|          1|
|    1|            O-2|             Army|          4.38|                 -0.51|            1|          1|
|    1|            O-3|             Army|     

In [10]:
#Account create date by month of the year
spark.sql( """SELECT count(id) , 
              Extract(Month From create_ddate) as month 
              From  dfAct_sql  
              group by month 
              ORDER BY month ASC """).show()

+---------+-----+
|count(id)|month|
+---------+-----+
|    11982|    1|
|     9626|    2|
|    10152|    3|
|    12579|    4|
|    10978|    5|
|    10672|    6|
|    10815|    7|
|    11912|    8|
|    10845|    9|
|    11866|   10|
|    12022|   11|
|     8992|   12|
+---------+-----+



In [11]:
#Registration for services by year
spark.sql( """SELECT count(id) , 
Extract(Year From create_ddate) as year_registered 
              From  dfAct_sql  
              group by year_registered
              ORDER BY year_registered ASC """).show()

+---------+---------------+
|count(id)|year_registered|
+---------+---------------+
|     2825|           2007|
|     2571|           2008|
|     1769|           2009|
|     2255|           2010|
|     2732|           2011|
|     4258|           2012|
|    13128|           2013|
|    13593|           2014|
|    13248|           2015|
|    25175|           2016|
|    22933|           2017|
|    26448|           2018|
|     1506|           2019|
+---------+---------------+



In [12]:
#Hired by Service branch
spark.sql(""" Select count(hires.id) as hired,bio.service_branch__c 
              From dfHire_sql hires 
              inner join dfBio_sql bio on bio.id = hires.id
              Where hires.hired = 1
              group by bio.service_branch__c
              order by hired asc
              """).show()

+-----+-----------------+
|hired|service_branch__c|
+-----+-----------------+
|    1|   Not Applicable|
|  170|      Coast Guard|
| 1145|             null|
| 2423|          Marines|
| 2879|        Air Force|
| 3055|             Navy|
|10417|             Army|
+-----+-----------------+



In [13]:
#Not hired by Service Branch
spark.sql(""" Select count(hires.id) as Not_hired,bio.service_branch__c 
              From dfHire_sql hires 
              inner join dfBio_sql bio on bio.id = hires.id
              Where hires.hired = 0
              group by bio.service_branch__c
              order by Not_hired asc
              """).show()

+---------+-----------------+
|Not_hired|service_branch__c|
+---------+-----------------+
|        2|   Not Applicable|
|      667|      Coast Guard|
|     9382|             null|
|    10524|        Air Force|
|    11260|          Marines|
|    12500|             Navy|
|    41314|             Army|
+---------+-----------------+



In [14]:
#Hired by rank
spark.sql(""" Select count(hires.id) as hired,bio.service_rank__c
              From dfHire_sql hires 
              inner join dfBio_sql bio on bio.id = hires.id
              Where hires.hired = 1
              group by bio.service_rank__c
              order by hired desc
              """).show()

+-----+---------------+
|hired|service_rank__c|
+-----+---------------+
| 3807|            E-5|
| 3683|            E-4|
| 2795|            E-7|
| 2775|            E-6|
| 1281|            E-8|
| 1156|           null|
| 1142|            O-3|
|  648|            E-3|
|  564|            O-4|
|  510|            O-5|
|  463|            E-9|
|  294|            O-2|
|  186|            O-6|
|  178|            W-3|
|  148|            W-4|
|  137|            W-2|
|  126|            E-2|
|   79|            O-1|
|   75|            E-1|
|   27|            W-5|
+-----+---------------+
only showing top 20 rows



In [15]:
#Prefered method of contact for the hired
spark.sql(""" select count(com.id) as count, com.preferred_method_of_contact__c 
              from dfCom_sql com 
              inner join dfHire_sql hires on com.id = hires.id
              where hires.hired = 1
              group by com.preferred_method_of_contact__c 
              order by count
          """).show()

+-----+------------------------------+
|count|preferred_method_of_contact__c|
+-----+------------------------------+
|    7|                          Mail|
|   23|                      LinkedIn|
|  158|                          Text|
| 1892|                          null|
| 2626|                        E-Mail|
|15384|                     Telephone|
+-----+------------------------------+



In [16]:
#Prefered method of contact for the Unhired
spark.sql(""" select count(com.id) as count, com.preferred_method_of_contact__c 
              from dfCom_sql com 
              inner join dfHire_sql hires on com.id = hires.id
              where hires.hired = 0
              group by com.preferred_method_of_contact__c 
              order by count
          """).show()

+-----+------------------------------+
|count|preferred_method_of_contact__c|
+-----+------------------------------+
|   28|                          Mail|
|   33|                      LinkedIn|
|  378|                          Text|
|12405|                        E-Mail|
|31506|                          null|
|41299|                     Telephone|
+-----+------------------------------+



In [17]:
#Select variables that determine whether one is going to be employed or not, store those varibles in a new dataframe
dfspark= spark.sql(""" select fact.yearsinservice as S_years , fact.reg_afterservice_years as R_years,
               act.resume_tailoring_tips__c as Resume_Tips,act.finalized_hhusa_revised_resume_on_file__c as Resume_OnFile, bio.client__c as Client, bio.client_type__c as Client_Type, bio.service_rank__c as Service_R
              ,bio.service_branch__c as Service_B, hire.hire_heroes_usa_confirmed_hire__c as HHUSA_hire, com.preferred_method_of_contact__c as Com_Method, com.responsive__c
              ,com.active__c, act.created_linkedin_account__c as Created_Linkedin, bio.highest_level_of_education_completed__c as Educ,  hire.hired
              ,bio.primary_military_occupational_specialty__c as Occupation
               
              from dfFact_sql fact
              
              inner join dfAct_sql act on act.id = fact.id
              inner join dfBio_sql bio on bio.id = fact.id
              inner join dfHire_sql hire on hire.id = fact.id
              inner join dfCom_sql com on com.id = fact.id
              
              where bio.client_type__c != '' and bio.service_branch__c !='' and bio.service_rank__c !='' 
              and com.preferred_method_of_contact__c !='' and fact.yearsinservice < 53 and fact.yearsinservice > 0 
              and fact.reg_afterservice_years >- 32 and fact.reg_afterservice_years < 53 and com.active__c !=''
              and bio.highest_level_of_education_completed__c !='' and hire.hired  !='' and hire.hired !='No' and hire.hired != 3

          """)

In [18]:
# Get the number of clients in the description above
dfspark.count()

51093

In [19]:
# Change the data type of the the hired column from string to double
dfspark = dfspark.withColumn("hired", dfspark["hired"].cast(DoubleType()))

In [20]:
# Confirm the labels in the hired column 0 represents the client was not hired 1 represents they were hired. 
dfspark.select('hired').distinct().show()

+-----+
|hired|
+-----+
|  0.0|
|  1.0|
+-----+



In [22]:
# The data types of the data frame 
dfspark.dtypes

[('S_years', 'double'),
 ('R_years', 'double'),
 ('Resume_Tips', 'int'),
 ('Resume_OnFile', 'int'),
 ('Client', 'int'),
 ('Client_Type', 'string'),
 ('Service_R', 'string'),
 ('Service_B', 'string'),
 ('HHUSA_hire', 'int'),
 ('Com_Method', 'string'),
 ('responsive__c', 'int'),
 ('active__c', 'string'),
 ('Created_Linkedin', 'int'),
 ('Educ', 'string'),
 ('hired', 'double'),
 ('Occupation', 'string')]

In [23]:
#Hash the features through a hash function to convert the features into vectors for the dfspark dataframe,
# Convert strings to numerical data, the output column is features. The target is not included in this conversion.
funchash = FeatureHasher(inputCols= ['S_years','R_years','Resume_Tips','Resume_OnFile','Client','Client_Type','Service_R',\
 'Service_B','HHUSA_hire','Com_Method','responsive__c','active__c','Created_Linkedin','Educ','Occupation'],
                       outputCol="features")

dffeatures = funchash.transform(dfspark)
dffeatures.show(1)

+-------+-------+-----------+-------------+------+-----------------+---------+---------+----------+----------+-------------+---------+----------------+---------------+-----+----------+--------------------+
|S_years|R_years|Resume_Tips|Resume_OnFile|Client|      Client_Type|Service_R|Service_B|HHUSA_hire|Com_Method|responsive__c|active__c|Created_Linkedin|           Educ|hired|Occupation|            features|
+-------+-------+-----------+-------------+------+-----------------+---------+---------+----------+----------+-------------+---------+----------------+---------------+-----+----------+--------------------+
|   8.11|  -0.37|          1|            1|     1|Online Registrant|      E-5|     Army|         0| Telephone|            1|      Yes|               1|High School/GED|  0.0|       91F|(262144,[52387,65...|
+-------+-------+-----------+-------------+------+-----------------+---------+---------+----------+----------+-------------+---------+----------------+---------------+-----+---

In [24]:
#Confirm that the dataframe is active, We are going to use this data to train our machine learning algorithm.
dffeatures.select('features','hired').show()

+--------------------+-----+
|            features|hired|
+--------------------+-----+
|(262144,[52387,65...|  0.0|
|(262144,[52387,65...|  0.0|
|(262144,[65085,69...|  0.0|
|(262144,[65085,69...|  0.0|
|(262144,[65085,69...|  0.0|
|(262144,[64366,65...|  0.0|
|(262144,[54411,65...|  0.0|
|(262144,[65085,69...|  0.0|
|(262144,[65085,69...|  0.0|
|(262144,[65085,69...|  0.0|
|(262144,[11074,65...|  0.0|
|(262144,[52387,65...|  0.0|
|(262144,[40267,65...|  0.0|
|(262144,[65085,69...|  0.0|
|(262144,[52387,65...|  0.0|
|(262144,[65085,69...|  0.0|
|(262144,[48884,52...|  0.0|
|(262144,[52387,65...|  0.0|
|(262144,[65085,69...|  0.0|
|(262144,[65085,69...|  0.0|
+--------------------+-----+
only showing top 20 rows



In [27]:
#Split the data into training and testing data for the training of the algorithms and testing of the algorithms 
train_data,test_data=dffeatures.randomSplit([0.75,0.25])

In [31]:
# Train the decision tree classifier, to predict whether one is hired or not, using the hired column and features. 
def treedec(label, features):
    decision_tree = DecisionTreeClassifier(labelCol=label, featuresCol=features)
    return decision_tree

In [33]:
# Now let us train the model, while timing how long it takes run this task. 

# Start time of training the model 
start = datetime.now()
def traindec_tree(train_data, algo): 
    # The model 
    model1 = algo.fit(train_data)
    return model1
# Print the duration
#traindec_tree(train_data, treedec('hired','features'))
#End of time of the model 
end = datetime.now()
print('Duration it takes to run the algorithm: {}'.format(end - start))

Duration it takes to run the algorithm: 0:00:00.000285


In [34]:
# Make predictions using the test set of the dataset ... Store the predictions in a new dataframe prediction 
predictions = traindec_tree(train_data, treedec('hired','features')).transform(test_data)

In [36]:
# Check out the new dataframe predictions to view the prediction, hired and features columns 
predictions.select("prediction", "hired", "features").show()

+----------+-----+--------------------+
|prediction|hired|            features|
+----------+-----+--------------------+
|       0.0|  0.0|(262144,[65085,81...|
|       1.0|  1.0|(262144,[52387,65...|
|       1.0|  1.0|(262144,[65085,69...|
|       0.0|  0.0|(262144,[65085,69...|
|       0.0|  0.0|(262144,[65085,69...|
|       0.0|  0.0|(262144,[65085,81...|
|       0.0|  0.0|(262144,[65085,69...|
|       0.0|  0.0|(262144,[6608,650...|
|       0.0|  0.0|(262144,[65085,69...|
|       0.0|  0.0|(262144,[65085,69...|
|       1.0|  1.0|(262144,[11074,65...|
|       0.0|  0.0|(262144,[65085,69...|
|       0.0|  0.0|(262144,[52387,65...|
|       0.0|  0.0|(262144,[63818,65...|
|       0.0|  0.0|(262144,[65085,69...|
|       0.0|  0.0|(262144,[65085,69...|
|       1.0|  1.0|(262144,[52387,65...|
|       1.0|  1.0|(262144,[65085,69...|
|       0.0|  0.0|(262144,[65085,69...|
|       0.0|  0.0|(262144,[11700,65...|
+----------+-----+--------------------+
only showing top 20 rows



In [43]:
# Now lets evaluate the algorithm performance
#predictions
def evaluate_algo(df,label,Algorithm_name  ):
    classification_evaluator = MulticlassClassificationEvaluator(
    labelCol=label, predictionCol="prediction")
    accuracy = classification_evaluator.evaluate(df)
    print(Algorithm_name )
    print("Test Error  = %g " % (1.0 - accuracy))
    print("Accuracy   = %g " % accuracy)

evaluate_algo(predictions,'hired','Decision Tree')

Decision Tree
Test Error  = 0.126202 
Accuracy   = 0.873798 


In [39]:
# Train a RandomForest model.
from datetime import datetime

start_time = datetime.now()
# do your work here
rf = RandomForestClassifier(labelCol="hired", featuresCol="features", numTrees=17)
end_time = datetime.now()

print('Duration: {}'.format(end_time - start_time))

def randomforest(label, features, numTrees):
    
    random_forest = RandomForestClassifier(labelCol=label, featuresCol=features, numTrees=numTrees)
    
    return random_forest

Duration: 0:00:00.007267


In [40]:
# Train model.  This also runs the indexers.

from datetime import datetime

start_time = datetime.now()
# do your work here
#modelrf = rf.fit(train_data)

def random_forestfit(randf,train_data):
    model= randf.fit(train_data)
    return model
    

end_time = datetime.now()

print('Duration: {}'.format(end_time - start_time))

Duration: 0:00:00.000230


In [41]:
# Make predictions.
predictions2 = random_forestfit(randomforest('hired', 'features', 17),train_data).transform(test_data)


In [42]:
# Select example rows to display.
predictions2.select('prediction','hired','features').show()  #.select("prediction", "indexedLabel", "features").show(5)

+----------+-----+--------------------+
|prediction|hired|            features|
+----------+-----+--------------------+
|       0.0|  0.0|(262144,[65085,81...|
|       0.0|  1.0|(262144,[52387,65...|
|       0.0|  1.0|(262144,[65085,69...|
|       0.0|  0.0|(262144,[65085,69...|
|       0.0|  0.0|(262144,[65085,69...|
|       0.0|  0.0|(262144,[65085,81...|
|       0.0|  0.0|(262144,[65085,69...|
|       0.0|  0.0|(262144,[6608,650...|
|       0.0|  0.0|(262144,[65085,69...|
|       0.0|  0.0|(262144,[65085,69...|
|       0.0|  1.0|(262144,[11074,65...|
|       0.0|  0.0|(262144,[65085,69...|
|       0.0|  0.0|(262144,[52387,65...|
|       0.0|  0.0|(262144,[63818,65...|
|       0.0|  0.0|(262144,[65085,69...|
|       0.0|  0.0|(262144,[65085,69...|
|       0.0|  1.0|(262144,[52387,65...|
|       0.0|  1.0|(262144,[65085,69...|
|       0.0|  0.0|(262144,[65085,69...|
|       0.0|  0.0|(262144,[11700,65...|
+----------+-----+--------------------+
only showing top 20 rows



In [44]:
evaluate_algo(predictions2,'hired','Random Forest')

Random Forest
Test Error  = 0.452746 
Accuracy   = 0.547254 


In [45]:
def cluster(k,df,features):
    kmeans = KMeans(k=k, seed=1)  # 2 clusters here
    model = kmeans.fit(df.select(features))
    return model

In [47]:
transformeddf = cluster(4,dffeatures,'features').transform(dffeatures)

In [48]:
transformeddf.select('prediction').distinct().show()

+----------+
|prediction|
+----------+
|         1|
|         3|
|         2|
|         0|
+----------+



In [50]:
#Information about individual clusters in term of years service men stay in the army and when they register for services
transformeddf.select('S_years','R_years').describe().show()

+-------+-----------------+------------------+
|summary|          S_years|           R_years|
+-------+-----------------+------------------+
|  count|            51093|             51093|
|   mean|13.25944708668499|1.9484418609202803|
| stddev| 8.55804960106253| 6.346271266872357|
|    min|             0.01|            -29.16|
|    max|            52.33|             52.58|
+-------+-----------------+------------------+



In [51]:
#Get cluster information Cluster 0 in terms of years service men spend on service. 
cluster0 = transformeddf.filter(transformeddf.prediction==0)

In [52]:
cluster0.select('S_years','R_years').describe().show()

+-------+------------------+------------------+
|summary|           S_years|           R_years|
+-------+------------------+------------------+
|  count|              2925|              2925|
|   mean| 5.224564102564105|22.217131623931618|
| stddev|2.9455794494751912| 8.277066032403162|
|    min|              0.01|             11.33|
|    max|             21.19|             52.58|
+-------+------------------+------------------+



In [53]:
#Get cluster information Cluster 1
cluster1 = transformeddf.filter(transformeddf.prediction==1)

In [54]:
cluster1.select('S_years','R_years').describe().show()

+-------+-----------------+-------------------+
|summary|          S_years|            R_years|
+-------+-----------------+-------------------+
|  count|            18935|              18935|
|   mean|22.77949405862156|-0.6843913387906007|
| stddev|4.482330273547443| 2.2575649893207146|
|    min|             14.0|             -29.16|
|    max|            52.33|               7.92|
+-------+-----------------+-------------------+



In [55]:
#Get cluster information Cluster 2
cluster2 = transformeddf.filter(transformeddf.prediction==2)

In [56]:
cluster2.select('S_years','R_years').describe().show()

+-------+------------------+-----------------+
|summary|           S_years|          R_years|
+-------+------------------+-----------------+
|  count|              1929|             1929|
|   mean|19.301316744427176|9.179803006739244|
| stddev| 4.354761181256989|4.399028957656696|
|    min|              8.92|             2.32|
|    max|             39.37|            26.82|
+-------+------------------+-----------------+



In [57]:
#Get cluster information Cluster 3
cluster3 = transformeddf.filter(transformeddf.prediction==3)

In [58]:
cluster3.select('S_years','R_years').describe().show()

+-------+------------------+------------------+
|summary|           S_years|           R_years|
+-------+------------------+------------------+
|  count|             27304|             27304|
|   mean| 7.091309698212723|1.0920648989159096|
| stddev|3.2336080116487533|3.1435676760042863|
|    min|              0.01|            -10.07|
|    max|             15.08|             11.83|
+-------+------------------+------------------+

